# Colmap container

In [15]:
import docker
serial = str(2)

path_to_mount = '/home/liteandfog/raspi-plant/d/dataset'+serial
path_masked_to_mount = '/home/liteandfog/raspi-plant/d2'
path_output_to_mount = '/home/liteandfog/raspi-plant/output3'

WD="/root/data"
destination_mount = '/root/data/dataset'
destination_masked_mount = '/root/data/dataset-m'
destination_output_mount = '/root/data/output'

mount_dict = {path_to_mount: {'bind': destination_mount, 'mode': 'rw'},
              path_masked_to_mount: {'bind': destination_masked_mount, 'mode': 'rw'},
              path_output_to_mount: {'bind': destination_output_mount, 'mode': 'rw'}}

#directories structure inside the docker container.
DATASET_PATH="/root/data"
DATABASE_PATH="/root/data/output/database.db"
IMAGESET_PATH="/root/data/dataset-m"
OUTPUT_PATH="/root/data/output"
SPARSE_PATH="/root/data/output/0"
NEW_SPARSE_PATH="/root/data/output/sparse"
DENSE_PATH="/root/data/output"
DENSE_PLY_PATH="/root/data/output/dense.ply"

# COLMAP 3D Reconstruction's pipeline

In [16]:
# feature_extractor
CMD1 = f"colmap feature_extractor --database_path {DATABASE_PATH} --image_path {IMAGESET_PATH}"
# exhaustive_matcher
CMD2 = f"colmap exhaustive_matcher \
   --database_path {DATABASE_PATH}"
# colmap mapper
CMD3 = f"colmap mapper \
    --database_path {DATABASE_PATH} \
    --image_path {IMAGESET_PATH} \
    --output_path {OUTPUT_PATH}"
# colmap image_undistorter
CMD4 = f"colmap image_undistorter \
    --image_path {IMAGESET_PATH} \
    --input_path {SPARSE_PATH} \
    --output_path {DENSE_PATH} \
    --output_type COLMAP"
# colmap patch_match_stereo
CMD5 = f"colmap patch_match_stereo \
    --workspace_path {DENSE_PATH} \
    --workspace_format COLMAP \
    --PatchMatchStereo.geom_consistency true"
# colmap stereo_fusion
CMD6 = f"colmap stereo_fusion \
    --workspace_path {DENSE_PATH} \
    --workspace_format COLMAP \
    --input_type geometric \
    --output_path {DENSE_PLY_PATH}"
### converter (bytes into text)
CMD7 = f"colmap model_converter \
    --input_path {SPARSE_PATH} \
    --output_path {SPARSE_PATH} \
    --output_type BIN"

In [17]:
%%time
client = docker.from_env()

CPU times: user 9.12 ms, sys: 231 µs, total: 9.35 ms
Wall time: 16.4 ms


In [18]:
%%time
client.containers.run('colmap:test', CMD1 ,volumes=mount_dict,working_dir=WD, runtime="nvidia",detach=False, auto_remove=True)

CPU times: user 54 ms, sys: 153 µs, total: 54.2 ms
Wall time: 50.7 s


b'\n==============================================================================\nFeature extraction\n==============================================================================\n\nProcessed file [1/180]\n  Name:            1641381121-832916/52img0-53.jpg\n  Dimensions:      720 x 1080\n  Camera:          #1 - SIMPLE_RADIAL\n  Focal Length:    1296.00px\n  Features:        1167\nProcessed file [2/180]\n  Name:            1641381121-832916/52img0-87.jpg\n  Dimensions:      720 x 1080\n  Camera:          #2 - SIMPLE_RADIAL\n  Focal Length:    1296.00px\n  Features:        1205\nProcessed file [3/180]\n  Name:            1641381121-832916/52img1-24.jpg\n  Dimensions:      720 x 1080\n  Camera:          #3 - SIMPLE_RADIAL\n  Focal Length:    1296.00px\n  Features:        1208\nProcessed file [4/180]\n  Name:            1641381121-832916/52img1-54.jpg\n  Dimensions:      720 x 1080\n  Camera:          #4 - SIMPLE_RADIAL\n  Focal Length:    1296.00px\n  Features:        1139\nProcessed 

In [19]:
%%time
client.containers.run('colmap:test', CMD2 ,volumes=mount_dict,working_dir=WD, runtime="nvidia",detach=False,auto_remove=True)

CPU times: user 21.1 ms, sys: 6.01 ms, total: 27.1 ms
Wall time: 1min 2s


b'\n==============================================================================\nExhaustive feature matching\n==============================================================================\n\nMatching block [1/4, 1/4] in 1.102s\nMatching block [1/4, 2/4] in 1.082s\nMatching block [1/4, 3/4] in 0.505s\nMatching block [1/4, 4/4] in 0.240s\nMatching block [2/4, 1/4] in 1.514s\nMatching block [2/4, 2/4] in 1.205s\nMatching block [2/4, 3/4] in 0.786s\nMatching block [2/4, 4/4] in 0.286s\nMatching block [3/4, 1/4] in 1.072s\nMatching block [3/4, 2/4] in 0.987s\nMatching block [3/4, 3/4] in 1.559s\nMatching block [3/4, 4/4] in 0.623s\nMatching block [4/4, 1/4] in 0.439s\nMatching block [4/4, 2/4] in 1.028s\nMatching block [4/4, 3/4] in 1.390s\nMatching block [4/4, 4/4] in 0.529s\nElapsed time: 1.024 [minutes]\n'

In [20]:
%%time
client.containers.run('colmap:test', CMD3 ,volumes=mount_dict,working_dir=WD, runtime="nvidia",detach=False,auto_remove=True)

CPU times: user 692 ms, sys: 24.3 ms, total: 717 ms
Wall time: 10min 49s


b"\n==============================================================================\nLoading database\n==============================================================================\n\nLoading cameras... 180 in 0.000s\nLoading matches... 5742 in 0.014s\nLoading images... 180 in 0.011s (connected 180)\nBuilding correspondence graph... in 0.030s (ignored 0)\n\nElapsed time: 0.001 [minutes]\n\n\n==============================================================================\nFinding good initial image pair\n==============================================================================\n\n\n==============================================================================\nInitializing with image pair #95 and #153\n==============================================================================\n\n\n==============================================================================\nGlobal bundle adjustment\n==============================================================================\n\niter      cos

In [21]:
%%time
client.containers.run('colmap:test', CMD4 ,volumes=mount_dict,working_dir=WD, runtime="nvidia",detach=False,auto_remove=True)

CPU times: user 33.6 ms, sys: 178 µs, total: 33.7 ms
Wall time: 2.44 s


b'\n==============================================================================\nReading reconstruction\n==============================================================================\n\n => Reconstruction with 162 images and 16118 points\n\n==============================================================================\nImage undistortion\n==============================================================================\n\nUndistorting image [1/162]\nUndistorting image [2/162]\nUndistorting image [3/162]\nUndistorting image [4/162]\nUndistorting image [5/162]\nUndistorting image [6/162]\nUndistorting image [7/162]\nUndistorting image [8/162]\nUndistorting image [9/162]\nUndistorting image [10/162]\nUndistorting image [11/162]\nUndistorting image [12/162]\nUndistorting image [13/162]\nUndistorting image [14/162]\nUndistorting image [15/162]\nUndistorting image [16/162]\nUndistorting image [17/162]\nUndistorting image [18/162]\nUndistorting image [19/162]\nUndistorting image [20/162]\nUn

In [22]:
%%time
client.containers.run('colmap:test', CMD5 ,volumes=mount_dict,working_dir=WD, runtime="nvidia",detach=False,auto_remove=True)

CPU times: user 550 ms, sys: 39.7 ms, total: 589 ms
Wall time: 1h 50s


b'Reading workspace...\nReading configuration...\nConfiguration has 162 problems...\n\n==============================================================================\nProcessing view 1 / 162 for 1641381121-832916/52img0-53.jpg\n==============================================================================\n\nReading inputs...\n\nPatchMatch::Problem\n-------------------\nref_image_idx: 102\nsrc_image_idxs: 17 115 153 152 73 142 141 95 150 149 74 143 97 144 148 99 145 151 105 147\n\nPatchMatchOptions\n-----------------\nmax_image_size: -1\ngpu_index: 0\ndepth_min: 3.01859\ndepth_max: 6.10677\nwindow_radius: 5\nwindow_step: 1\nsigma_spatial: 5\nsigma_color: 0.2\nnum_samples: 15\nncc_sigma: 0.6\nmin_triangulation_angle: 1\nincident_angle_sigma: 0.9\nnum_iterations: 5\ngeom_consistency: 0\ngeom_consistency_regularizer: 0.3\ngeom_consistency_max_cost: 3\nfilter: 0\nfilter_min_ncc: 0.1\nfilter_min_triangulation_angle: 3\nfilter_min_num_consistent: 2\nfilter_geom_consistency_max_cost: 1\nwrite

In [23]:
%%time
client.containers.run('colmap:test', CMD6 ,volumes=mount_dict,working_dir=WD, runtime="nvidia",detach=False,auto_remove=True)

CPU times: user 30.1 ms, sys: 1.29 ms, total: 31.4 ms
Wall time: 29.8 s


b'\nStereoFusion::Options\n---------------------\nmask_path: \nmax_image_size: -1\nmin_num_pixels: 5\nmax_num_pixels: 10000\nmax_traversal_depth: 100\nmax_reproj_error: 2\nmax_depth_error: 0.01\nmax_normal_error: 10\ncheck_num_images: 50\nuse_cache: 0\ncache_size: 32\nbbox_min: -3.40282e+38 -3.40282e+38 -3.40282e+38\nbbox_max: 3.40282e+38 3.40282e+38 3.40282e+38\n\nReading workspace...\nLoading workspace data with 16 threads...\nElapsed time: 0.015 [minutes]\nReading configuration...\nStarting fusion with 16 threads\nFusing image [1/162] with index 0 in 0.289s (16135 points)\nFusing image [2/162] with index 106 in 0.224s (24763 points)\nFusing image [3/162] with index 109 in 0.221s (31459 points)\nFusing image [4/162] with index 111 in 0.222s (37186 points)\nFusing image [5/162] with index 113 in 0.226s (42273 points)\nFusing image [6/162] with index 116 in 0.229s (46852 points)\nFusing image [7/162] with index 72 in 0.227s (50852 points)\nFusing image [8/162] with index 71 in 0.229s (

In [24]:
%%time
client.containers.run('colmap:test', CMD7 ,volumes=mount_dict,working_dir=WD, runtime="nvidia",detach=False)

CPU times: user 23.6 ms, sys: 3.64 ms, total: 27.3 ms
Wall time: 700 ms


b''

#  Visualization

In [25]:
import numpy as np
import open3d as o3d
import vedo

#embedded vedo in jupyter
vedo.settings.embedWindow(backend='ipyvtk', verbose=True)

In [26]:
local_path_to_dense_pc = "/home/liteandfog/Desktop/raspi-plant/output3/dense.ply"

In [13]:
#%%time
#o3d.visualization.draw_geometries([o3d.io.read_point_cloud(local_path_to_dense_pc)])  # Visualize the point cloud

In [28]:
vedo.show(local_path_to_dense_pc, axes=True)

ViewInteractiveWidget(height=720, layout=Layout(height='auto', width='100%'), width=720)

####

In [15]:
##  Visualization: Poisson's mesh

In [32]:
%%time
cloud = o3d.io.read_point_cloud(local_path_to_dense_pc)
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(cloud, depth=12, width=0, scale=1.1, linear_fit=False)[0]

CPU times: user 3min 3s, sys: 30 s, total: 3min 34s
Wall time: 21.1 s


[WARNING] /home/liteandfog/Open3D/build/poisson/src/ext_poisson/PoissonRecon/Src/FEMTree.IsoSurface.specialized.inl (Line 1858)
          Extract
          bad average roots: 4


In [33]:
local_path_to_poison_mesh = f"/home/liteandfog/raspi-plant/output/poisson.ply"
o3d.io.write_triangle_mesh(local_path_to_poison_mesh, poisson_mesh, write_ascii=True, compressed=False)

[Open3D WARNING] Write Ply clamped color value to valid range


True

In [31]:
vedo.show(local_path_to_poison_mesh)

ViewInteractiveWidget(height=720, layout=Layout(height='auto', width='100%'), width=720)

# Outliers and isolated points removal.

In [34]:
def create_outliers_cloud(pcd):
    pcd_colors = np.asarray(pcd.colors)*255
    pcd_colors_summed = np.expand_dims(pcd_colors.sum(axis=1), axis=1)
    outliers_ind = np.where(np.any(pcd_colors_summed > 720,axis = 1))[0].tolist()
    return pcd.select_by_index(outliers_ind,invert=True),pcd.select_by_index(outliers_ind,invert=False)

In [45]:
inliers_cloud, outliers_cloud = create_outliers_cloud(cloud)

In [47]:
o3d.visualization.draw_geometries([inliers_cloud])

In [37]:
local_path_to_inliers_cloud = f"/home/liteandfog/raspi-plant/output/inliers_cloud.ply"
local_path_to_outliers_cloud = f"/home/liteandfog/raspi-plant/output/outliers_cloud.ply"
local_path_to_inliers_xpcd_cloud = f"/home/liteandfog/raspi-plant/output/inliers_xpcd_cloud.ply"

o3d.io.write_point_cloud(local_path_to_inliers_cloud, inliers_cloud, write_ascii=True, compressed=False)
o3d.io.write_point_cloud(local_path_to_outliers_cloud, outliers_cloud, write_ascii=True, compressed=False)
xpcd, _ = inliers_cloud.remove_statistical_outlier(nb_neighbors=5,std_ratio=2.0)

In [39]:
vedo.show(local_path_to_inliers_cloud)

ViewInteractiveWidget(height=720, layout=Layout(height='auto', width='100%'), width=720)

In [40]:
vedo.show(local_path_to_outliers_cloud)

ViewInteractiveWidget(height=720, layout=Layout(height='auto', width='100%'), width=720)

In [43]:
#o3d.visualization.draw_geometries([poisson_mesh])
#o3d.visualization.draw_geometries([xpcd])
#o3d.visualization.draw_geometries([outliers_cloud])
#o3d.visualization.draw_geometries([inliers_cloud])

# Poisson mesher - cleansed cloud

In [41]:
poisson_mesh_cleansed = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(xpcd, depth=14, width=0, scale=1.1, linear_fit=False)[0]
local_path_to_poisson_mesh_cleansed = f"/home/liteandfog/raspi-plant/output/poisson_cleansed.ply"
o3d.io.write_triangle_mesh(local_path_to_poisson_mesh_cleansed, poisson_mesh_cleansed, write_ascii=True, compressed=False)

True

In [42]:
vedo.show(local_path_to_poisson_mesh_cleansed)

ViewInteractiveWidget(height=720, layout=Layout(height='auto', width='100%'), width=720)